In [1]:
import pandas as pd

column_names = pd.read_csv('column_names.csv', header=None)
dataset = pd.read_csv('chronic_dataset.csv', header=None)
dataset = dataset.iloc[:,:25]
dataset.columns = column_names[0]

In [2]:
dataset = dataset[dataset != '?']
dataset = dataset.dropna()

In [3]:
def gnumeric_func (df, column_list):
    df[column_list] = df[column_list].apply(lambda x: pd.factorize(x)[0])
    return df

In [4]:
df1 = gnumeric_func(dataset, ['rbc','pc','pcc','ba','htn','dm','cad','appet','pe','ane','class'])

In [5]:
df1.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
3,48,70,1.005,4,0,0,0,0,0,117,...,32,6700,3.9,0,0,0,0,0,0,0
9,53,90,1.02,2,0,1,0,0,0,70,...,29,12100,3.7,0,1,0,0,1,0,0
11,63,70,1.01,3,0,1,0,0,0,380,...,32,4500,3.8,0,1,0,0,0,1,0
14,68,80,1.01,3,2,0,0,0,1,157,...,16,11000,2.6,0,1,1,0,0,1,0
20,61,80,1.015,2,0,1,0,1,0,173,...,24,9200,3.2,0,1,1,0,0,0,0


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, cross_validation
predicted = cross_validation.cross_val_predict(LogisticRegression(), df1.iloc[:,:24], df1['class'], cv=2)
print (metrics.accuracy_score(df1['class'], predicted))

0.9936305732484076


/home/girish/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
print (metrics.classification_report(df1['class'], predicted) )

             precision    recall  f1-score   support

          0       1.00      0.98      0.99        43
          1       0.99      1.00      1.00       114

avg / total       0.99      0.99      0.99       157



In [8]:
clf= LogisticRegression()
model = clf.fit(df1.iloc[:,:24], df1['class'])

In [9]:
model.coef_

array([[-1.75107494e-01,  6.07822394e-02, -8.86938182e-03,
        -5.34443345e-01, -3.86831332e-03, -2.13977810e-02,
         1.54232975e-01, -2.49837121e-03, -1.16622533e-02,
        -1.60190020e-01, -2.02357696e-01, -1.58709880e-01,
         7.88255887e-02, -7.43398071e-02,  3.39696232e-01,
         6.29523450e-01, -5.85610499e-04,  2.01493389e-01,
         1.44786046e-01, -1.50308003e-01, -1.84589710e-05,
         1.91480645e-03, -3.75860248e-03, -2.63621616e-03]])

In [10]:
model.coef_.argsort()[0][0:5]

array([ 3, 10,  0,  9, 11])

In [11]:
model.coef_.argsort()[0][-5:]

array([18,  6, 17, 14, 15])

In [12]:
dataset.columns[model.coef_.argsort()[0][0:5]]

Index(['al', 'bu', 'age', 'bgr', 'sc'], dtype='object', name=0)

In [13]:
dataset.columns[model.coef_.argsort()[0][-5:]]

Index(['htn', 'pc', 'rbcc', 'hemo', 'pcv'], dtype='object', name=0)

In [15]:
df2 = df1[['age','al','bp','bu','dm','hemo','htn','pc','rbcc','su']]

In [16]:
df2.shape

(157, 10)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, cross_validation
predicted = cross_validation.cross_val_predict(LogisticRegression(), df2, df1['class'], cv=2)
print (metrics.accuracy_score(df1['class'], predicted))

0.9936305732484076


In [18]:
print (metrics.classification_report(df1['class'], predicted) )

             precision    recall  f1-score   support

          0       1.00      0.98      0.99        43
          1       0.99      1.00      1.00       114

avg / total       0.99      0.99      0.99       157



In [20]:
from sklearn.linear_model import LogisticRegression
clf= LogisticRegression()
model = clf.fit(df2, df1['class'])
model.coef_

array([[-0.06715055, -1.76029909, -0.06507503, -0.04185037, -0.45054112,
         0.86355226,  0.31403459,  0.12783379,  0.33702823, -0.56755321]])

In [9]:
import pickle
filename = 'chronic_kidney_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [10]:
main_dict = {"age":"43", "albumin":"3", "bloodPressure":"23", "bloodUrea":"76", "dm":"1", "hemoglobin":"67","htn":"0","pc":"0","rbcc":"98","sugar":"1" }
json_dict = {k: [int(v)] for k, v in main_dict.items()}
df2= pd.DataFrame.from_dict(data=json_dict)

In [17]:
float(model.predict_proba(df1.ix[250])[0][0])

/home/girish/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


0.0014124876054041335